In [1]:
import logging
from datetime import datetime, timedelta, timezone
from nost_tools import application
from nost_tools.application_utils import ConnectionConfig, ShutDownObserver
from dotenv import dotenv_values
import time

# Define the callback function
def message_callback(ch, method, properties, body):
    print(f"Received message: {body}")

def callback(ch, method, properties, body):
    print(f" [x] {method.routing_key}:{body.decode('utf-8')}")

# Set up logging
logging.basicConfig(level=logging.INFO)

# Set the prefix=exchange
PREFIX = 'nost'

# Load credentials from a .env file in current working directory
credentials = dotenv_values(".env")
HOST, PORT = credentials["HOST"], int(credentials["PORT"])
USERNAME, PASSWORD = credentials["USERNAME"], credentials["PASSWORD"]
CLIENT_ID = credentials["CLIENT_ID"]
CLIENT_SECRET_KEY = credentials["CLIENT_SECRET_KEY"]
VIRTUAL_HOST = credentials["VIRTUAL_HOST"]
IS_TLS = credentials["IS_TLS"].lower() == 'true'  # Convert to boolean

# Set the client credentials from the config file
config = ConnectionConfig(
    USERNAME,
    PASSWORD,
    HOST,
    PORT,
    CLIENT_ID,
    CLIENT_SECRET_KEY,
    VIRTUAL_HOST,
    IS_TLS)

# create the manager application from the template in the tools library
app = application.Application(app_name='test')

# add a shutdown observer to shut down after a single test case
app.simulator.add_observer(ShutDownObserver(app))

# start up the manager on PREFIX from config file
app.start_up(PREFIX, config, True)

# Send ready message
app.ready()

# Add the callback to the application
app.add_message_callback(app_name='test', app_topic='status.time', callback=callback)

# # Additional logging
# logging.info("Application started and callback added.")

# # Test the remove_message_callback function
# def test_remove_message_callback():
#     # Add the callback
#     app.add_message_callback(app_name='test', app_topic='status.time', callback=callback)
#     logging.info("Callback added for testing removal.")

#     # Remove the callback
#     app.remove_message_callback(app_name='test', app_topic='status.time')
#     logging.info("Callback removed for testing.")

#     # Verify the callback removal
#     # You can send a test message to the topic and check that the callback is not executed
#     # This part depends on your messaging system and how you can send test messages

# # Run the test
# test_remove_message_callback()

# while True:
#     time.sleep(1)


INFO:nost_tools.application:Contacting pool.ntp.org to retrieve wallclock offset.
INFO:nost_tools.application:Wallclock offset updated to 0:00:00.003475.
INFO:nost_tools.application:Access token successfully acquired.
INFO:pika.adapters.utils.connection_workflow:Pika version 1.3.2 connecting to ('127.0.0.1', 5672)
INFO:pika.adapters.utils.io_services_utils:Socket connected: <socket.socket fd=75, family=2, type=1, proto=6, laddr=('127.0.0.1', 59504), raddr=('127.0.0.1', 5672)>
INFO:pika.adapters.utils.connection_workflow:Streaming transport linked up: (<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7f872df369c0>, _StreamingProtocolShim: <SelectConnection PROTOCOL transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7f872df369c0> params=<ConnectionParameters host=localhost port=5672 virtual_host=/ ssl=False>>).
INFO:pika.adapters.utils.connection_workflow:AMQPConnector - reporting success: <SelectConnection OPEN transport=<pik

In [2]:
# app.channel.start_consuming()

In [3]:
app.remove_message_callback(app_name='test', app_topic='status.time')